In [2]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
import os

#給予可以編輯的權限
scopes = ['https://www.googleapis.com/auth/calendar']

flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)
credentials_user = flow.run_console()

#將使用者的權限資料存進去token_user.pkl裡面
pickle.dump(credentials_user, open("token_user.pkl", "wb"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=389104841980-4nhfdf702k6na9huv3qke0lsn18cqj9m.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=P5XRMH02GGXv3Vfclfe7VXaQKkbmDU&prompt=consent&access_type=offline
Enter the authorization code: 4/wQHBfGR-BLoPJjMznroNFPMIsCI5RHMSYUSt6tchgDPrD6Bjff-PEJQ


In [4]:
#把user資料拿出來用
credential_receiver = pickle.load(open("./token_user.pkl" , "rb"))
service_receive = build("calendar", "v3", credentials = credential_receiver)

#把owner資料阿出來用
credential_sender = pickle.load(open("./token_owner.pkl" , "rb"))
service_send = build("calendar", "v3", credentials = credential_sender)   

#將owner google calendar 中所有行事曆列出來
senderCalenderList = service_send.calendarList().list().execute()

#找出owner google calendar 中叫做"2年24班"的行事曆
for calendar in senderCalenderList["items"]:
    if calendar["summary"] == "2年24班":
        calendarId = calendar["id"]

#receiverId就是user的帳號
receiverId = service_receive.calendarList().get(calendarId = "primary").execute()["id"]

#對於"2年24班"這個行事曆要給user多大的權限
#writer：更改的所有權
new_acl_rule = {
        'scope': {
        'type': 'user',
        'value': receiverId,
    },
'role': 'writer'
}

#執行共用這個步驟
target_response = service_send.acl().insert(calendarId = calendarId, body = new_acl_rule, sendNotifications = True).execute()